In [59]:
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#from scipy.stats import shapiro, kstest, poisson, chisquare, ttest_ind, levene, bartlett, sem, ppf
import scipy.stats as stats
from scipy.stats import shapiro, levene
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

****Fase 1: Exploración y Limpieza****

1.1. Exploración inicial

Cargamos los datos

In [60]:
df = pd.read_csv('data/final.csv', index_col=0)
df.head(2)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,680565,Canada,Quebec,Montreal,H2T 2J6,Male,Bachelor,83194.0,Married,Star,4462.45,Standard,2013,6,NaN,NaN,2017,1,0,0,0,0,0.0,0,0
1,680565,Canada,Quebec,Montreal,H2T 2J6,Male,Bachelor,83194.0,Married,Star,4462.45,Standard,2013,6,NaN,NaN,2017,2,8,0,8,784,78.0,0,0


Hacemos una exploración de los datos

In [ ]:
def exploracion(dataframe):
    """
    Realiza un análisis exploratorio básico sobre un DataFrame de pandas y muestra información estadística.

    Esta función proporciona un resumen del DataFrame, incluyendo el porcentaje de valores nulos y no nulos, 
    el tipo de dato de cada columna, el número de valores únicos en cada columna, y los principales estadísticos 
    de las columnas categóricas y numéricas.

    Parámetros:
    ----------
    dataframe : pandas.DataFrame
        El DataFrame sobre el cual se realizará la exploración. Debe ser un objeto pandas que contenga datos 
        organizados en filas y columnas.

    Salida:
    -------
    None
        La función imprime directamente el análisis del DataFrame sin retornar ningún valor.
    
    Proceso:
    --------
    - Calcula el porcentaje de valores nulos y no nulos para cada columna.
    - Muestra el tipo de dato de cada columna.
    - Calcula y muestra el número de valores únicos por columna.
    - Imprime las dimensiones del DataFrame (número de filas y columnas).
    - Muestra un resumen estadístico de las columnas categóricas, si existen.
    - Muestra un resumen estadístico de las columnas numéricas, si existen.
    
    Ejemplo:
    --------
    # Supongamos que df es un DataFrame previamente cargado.
    exploracion(df)

    Esto mostrará el porcentaje de nulos, el tipo de datos, los valores únicos y los estadísticos descriptivos 
    de las columnas categóricas y numéricas del DataFrame.
    """
    df_info = pd.DataFrame() # Vamos a ir creando este DF que contendrá la información estadística que vamos a ir calculando a continuación:
    # Añadimos una columna a nuestro df_info que nos da el % de nulos por campo, relacionando la suma de nulos (isna) con el número de datos totales que contiene el la columna
    # redonde el resultado a 2 decimales
    df_info["% nulos"] = round(dataframe.isna().sum()/dataframe.shape[0]*100, 2) 
    # Añadimos otra columna que relaciona los no nulos de la columna con el total de datos que contiene, redondeando los decimales    
    df_info["% no_nulos"] = round(dataframe.notna().sum()/dataframe.shape[0]*100, 2)
    # Otra columna con que nos informa del tipo de dato de cada columna
    df_info["tipo_dato"] = dataframe.dtypes 
    # Otra con los valores únicos
    df_info["num_valores_unicos"] = dataframe.nunique() 

      
    # imprime las filas, columnas
    print(f"El DataFrame tiene {dataframe.shape[0]} filas y {dataframe.shape[1]} columnas.") 
    
    #imprime la información que hemos ido recogiendo en el df
    display(df_info)

    # Imprime los principales valores estadísticos, separando la información por columnas categóricas y numéricas
    # el try except es para evitar el ValueError que nos da pq en df1 no hay columnas categóricas. Y ya lo ampliamos para las columnas numéricas
    print("Principales estadísticos de las columnas categóricas:")
    try:
        display(dataframe.describe(include="O").T)
    except ValueError:
        print ('No existen columnas categóricas')

    print("Principales estadísticos de las columnas numéricas:")
    try:
        display(dataframe.describe(exclude="O").T)
    except ValueError:
        print ('No existen columnas numéricas')

    return 

In [62]:
exploracion(df)

El DataFrame tiene 48408 filas y 25 columnas.


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.00,100.00,int64,2000
Country,0.00,100.00,object,1
Province,0.00,100.00,object,11
City,0.00,100.00,object,29
Postal Code,0.00,100.00,object,54
Gender,0.00,100.00,object,2
Education,0.00,100.00,object,5
Salary,25.48,74.52,float64,1297
Marital Status,0.00,100.00,object,3
Loyalty Card,0.00,100.00,object,3


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Country,48408,1,Canada,48408
Province,48408,11,Ontario,16104
City,48408,29,Toronto,10104
Postal Code,48408,54,V6E 3D9,2544
Gender,48408,2,Female,25008
Education,48408,5,Bachelor,30528
Marital Status,48408,3,Married,27600
Loyalty Card,48408,3,Star,20880
Enrollment Type,48408,2,Standard,45552


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,48408.0,557202.133366,256727.139690,100018.00,338346.00,561251.00,780944.00,998776.00
Salary,36072.0,78752.049900,33751.855593,-26322.00,59262.00,73871.00,87867.00,407228.00
CLV,48408.0,8008.344080,6936.643492,1898.01,4018.88,5815.26,8972.14,83325.38
Enrollment Year,48408.0,2015.294993,1.996731,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,48408.0,6.821021,3.339704,1.00,4.00,7.00,10.00,12.00
Cancellation Year,6456.0,2016.505576,1.373298,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,6456.0,7.063197,3.533402,1.00,4.00,7.00,10.00,12.00
Year,48408.0,2017.500000,0.500005,2017.00,2017.00,2017.50,2018.00,2018.00
Month,48408.0,6.500000,3.452088,1.00,3.75,6.50,9.25,12.00
Flights Booked,48408.0,4.054165,5.225294,0.00,0.00,1.00,8.00,21.00


In [63]:
df.columns

Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month', 'Year', 'Month',
       'Flights Booked', 'Flights with Companions', 'Total Flights',
       'Distance', 'Points Accumulated', 'Points Redeemed',
       'Dollar Cost Points Redeemed'],
      dtype='object')

In [64]:
#vemos todos los valores ordenados
df[['Loyalty Number','Year', 'Month',
       'Flights Booked', 'Flights with Companions', 'Total Flights',
       'Distance', 'Points Accumulated', 'Points Redeemed',
       'Dollar Cost Points Redeemed']].sort_values(by=['Loyalty Number', 'Year', 'Month']).head(2)
# Vemos que los clientes tienen registro mensual hayan o no volado

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
22128,100018,2017,1,3,0,3,1521,152.0,0,0
22129,100018,2017,2,2,2,4,1320,132.0,0,0


In [65]:
print(f'df1 Tiene {df.duplicated().sum()} datos duplicados, lo que supone un porcentaje de {round(df.duplicated().sum()/df.shape[0]*100, 2)}% de los datos.')

df1 Tiene 153 datos duplicados, lo que supone un porcentaje de 0.32% de los datos.


In [66]:
# Revisamos los duplicados
duplicados = df[df.duplicated(keep=False)].sort_values(by=['Loyalty Number', 'Year', 'Month'], ascending=True) .head(50)
## Están duplicados porque se cada fila de datos se refiere a un año y un mes, pero están vacíos de datos relevantes
## Nos vamos a quedar solo con uno de ellos para tener dato de ese mes y año de ese cliente, aunque esté vacío

duplicados.head(2)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
48360,106509,Canada,Ontario,Toronto,M8Y 4K8,Male,Bachelor,54529.0,Married,Nova,16727.77,Standard,2017,4,2018.0,10.0,2017,1,0,0,0,0,0.0,0,0
48361,106509,Canada,Ontario,Toronto,M8Y 4K8,Male,Bachelor,54529.0,Married,Nova,16727.77,Standard,2017,4,2018.0,10.0,2017,1,0,0,0,0,0.0,0,0


1.2. Limpieza de datos

Limpiamos duplicados

In [68]:
# Vemos en número de filas antes de borrar duplicados ((48408, 25))
df.shape 

(48408, 25)

In [69]:
df=df.drop_duplicates(duplicados)

In [70]:
df.shape # despues de borrar duplicados (48255, 25), la diferencia son los 153 duplicados que teníamos

(48255, 25)

In [71]:
## En las consultas con los csv anteriores tenian duplicados los registros, uno con los vuelos con acompañantes y otro sin ellos
# En este, no aparecen. Esto es porque el César ha hecho un inner y ha sumado solito
consulta_antes_gb=df[df['Loyalty Number'] == 423254].sort_values(by=[ 'Year', 'Month'], ascending=True)
consulta_antes_gb.head(2)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
624,423254,Canada,Ontario,Ottawa,K1F 2R2,Male,Doctor,70755.0,Divorced,Aurora,11315.2,Standard,2015,8,NaN,NaN,2017,1,0,0,0,0,0.0,0,0
625,423254,Canada,Ontario,Ottawa,K1F 2R2,Male,Doctor,70755.0,Divorced,Aurora,11315.2,Standard,2015,8,NaN,NaN,2017,2,7,0,7,805,80.0,0,0


Limpiamos valores negativos en Salary


In [72]:
df[df['Salary'] <0].head(2)
# Es solo el de 1 cliente. Entendemos que es un error al teclear. No hay salarios negativos. Lo sustituimos por el mismo valor positivo

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
18816,364596,Canada,Quebec,Tremblant,H5Y 2S9,Female,Bachelor,-26322.0,Single,Aurora,16710.84,2018 Promotion,2018,4,2018.0,12.0,2017,1,0,0,0,0,0.0,0,0
18817,364596,Canada,Quebec,Tremblant,H5Y 2S9,Female,Bachelor,-26322.0,Single,Aurora,16710.84,2018 Promotion,2018,4,2018.0,12.0,2017,2,0,0,0,0,0.0,0,0


In [73]:
#Quitamos los valores negativos
df['Salary'] = df['Salary'].abs()
df['Salary'].min() # Para comprobar que ya no hay negativos

20173.0

Tratamiento de nulos

In [ ]:
# Tenemos nulos en:
# - Cancellation Year y Cancellation Month. No hacemos nada porque ese nulo significa que el cliente no está dado de baja
# - En Salary tenemos un 25.32% de los datos nulos y es un dato que necesitamos para el estudio, con lo que vamos a intentar sustituirlos por la media
#   A ver si no cambian mucho nuestros datos

In [ ]:
# Comprobamos nulos antes de fillna
df['Salary'].isna().sum() #--> 12319

12319

In [ ]:
def nulos_x_media(columna):
    """
    Sustituye los valores nulos de una columna en un DataFrame por su media y muestra información sobre el cambio.

    Esta función calcula la media de una columna antes y después de sustituir los valores nulos por la media de la columna.
    Imprime información sobre la cantidad de valores nulos en la columna antes de la sustitución, la media antes de la sustitución, 
    y la nueva media después de realizar la sustitución.

    Parámetros:
    ----------
    columna : str
        El nombre de la columna en el DataFrame sobre la cual se realizará la sustitución de valores nulos por la media.

    Salida:
    -------
    None
        La función no devuelve ningún valor. Imprime directamente la información sobre la columna y el proceso de sustitución.

    Ejemplo:
    --------
    # Supongamos que 'edad' es una columna con valores nulos en el DataFrame df.
    nulos_x_media('edad')

    Esto imprimirá la media de la columna 'edad' antes y después de reemplazar los valores nulos con la media.
    """
    print(f' La media de {columna} antes de la sustitución de nulos es {df[columna].mean()}')
    print(f'Vamos a sustituir {df[columna].isnull().sum()} nulos de {columna} por la media')
    df[columna] = df[columna].fillna(df[columna].mean())    
    print(f'Después de la sustitución, la nueva media de {columna} es {df[columna].mean()}')

In [77]:
nulos_x_media('Salary')

 La media de Salary antes de la sustitución de nulos es 78782.82287956367
Vamos a sustituir 12319 nulos de Salary por la media
Después de la sustitución, la nueva media de Salary es 78782.82287956367


In [ ]:
# Comprobamos nulos despues de fillna
df['Salary'].isna().sum()

0

Guardo los datos limpios en un csv nuevo

In [79]:
df.to_csv("data/final_limpio.csv")